### QM9 Model
Model trained on 2000 QM9 molecules from Figure 2 of https://arxiv.org/abs/2407.20384. Prediction of C4H6N4O

In [1]:
import joblib
gnn_obj_file = 'qm9_model.joblib'
gnn_obj = joblib.load(gnn_obj_file)

# predict sigma(iw) in the SAIAO basis froma chkfile (this is a training example from workflow_gnn output)
example_chkfile = '../workflow_gnn/test_chk/ammonia.chk'
sigma_ml = gnn_obj.predict_full_sigma(example_chkfile) 


Apr 08 16:46:53 ####### 
                ####### libDMET   version 0.5
                ####### A periodic DMET library for lattice model and realistic solid.
                ####### 


In [2]:
# the pdset attribute contains the Moldatum object that was mounted for an MBGF-Net prediction
mlf = gnn_obj.pdset[0]

# extract the SAIAO to MO rotation for sigma(iw)
C_saiao_mo = mlf['C_saiao_mo']

# rotate sigma(iw) to MO basis
from mlgf.lib.ml_helper import sigma_lo_mo
sigma_ml_mo = sigma_lo_mo(sigma_ml, C_saiao_mo) 

# extract the properties from MBGF and self-energy, takes sigma in the MO basis, can be machine learned sigma or true self-energy
from mlgf.workflow.get_ml_info import get_properties
import numpy as np
eta = 0.01 # band broadening for DOS
freqs = np.linspace(-1, 1, 201) # real frequency points on which to evaluate the DOS
properties = 'dqmb' # a short string denoting which properties to compute from sigma: d - dos, q - qpe, m - density matrix, b - bse

# the indicies of the sigmaI points used for analytic continuation for QPE
ac_idx = [ 0,  2,  3,  5,  6,  8,  9, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23]
properties_ml = get_properties(sigma_ml_mo, mlf, freqs, eta, properties = properties, ac_idx = ac_idx)
properties_true = get_properties(mlf['sigmaI'], mlf, freqs, eta, properties = properties, ac_idx = ac_idx)

In [3]:
au_to_ev = 27.21
nocc = mlf['nocc']

homo_error = properties_ml['qpe'][nocc-1] - properties_true['qpe'][nocc-1]
print(f'HOMO Error (eV): {homo_error*au_to_ev:0.4f}') 

lumo_error = properties_ml['qpe'][nocc] - properties_true['qpe'][nocc]
print(f'LUMO Error (eV): {lumo_error*au_to_ev:0.4f}') 

dm_mae = np.mean(np.abs(properties_ml['dm'] - properties_true['dm']))
print(f'Density matrix MAE: {dm_mae:0.4f}') 

# dipoles and quadrupoles, note predicted dm is in MO basis
from mlgf.lib.dm_helper import dm_mo_to_ao, get_dipole, scalar_quadrupole
from pyscf import dft, lib
mol = lib.chkfile.load_mol(example_chkfile)
rks = dft.RKS(mol)
rks.xc = mlf['xc'].decode('utf-8')
scf_data = lib.chkfile.load(example_chkfile, 'scf')
rks.__dict__.update(scf_data)
dipole_ml = get_dipole(rks, dm_mo_to_ao(properties_ml['dm'], scf_data['mo_coeff']))
dipole_true = get_dipole(rks, dm_mo_to_ao(properties_true['dm'], scf_data['mo_coeff']))

quadrupole_ml = scalar_quadrupole(mol, dm_mo_to_ao(properties_ml['dm'], scf_data['mo_coeff']))
quadrupole_true = scalar_quadrupole(mol, dm_mo_to_ao(properties_true['dm'], scf_data['mo_coeff']))

print(f'Dipole error (Debye): {(dipole_ml-dipole_true):0.4f}') 
print(f'Quadrupole error (Debye⋅Å): {(quadrupole_ml-quadrupole_true):0.4f}') 

if 'b' in properties:
    s1_ml = properties_ml['bse_exci_s'][0]*au_to_ev
    s1_true = properties_true['bse_exci_s'][0]*au_to_ev
    print(f'BSE S1 error (eV): {(s1_ml-s1_true):0.4f}') 
    

HOMO Error (eV): 0.0037
LUMO Error (eV): 0.0073
Density matrix MAE: 0.0000
Dipole error (Debye): 0.0057
Quadrupole error (Debye⋅Å): 0.0007
BSE S1 error (eV): 0.0039
